In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
x = np.load('/dbfs/FileStore/tables/npy/augmented_merged_fdataX_3.npy')
y = np.load('/dbfs/FileStore/tables/npy/augmented_merged_flabels_3.npy')

In [11]:
x = x/255.0

x_valid, x_train, y_valid, y_train = train_test_split(x, y, test_size=0.80, random_state=41)
x_valid, x_test, y_valid, y_test = train_test_split(x_valid, y_valid, test_size=0.50, random_state=42)

train_x = tf.convert_to_tensor(x_train, np.float32)
test_x = tf.convert_to_tensor(x_test, np.float32)

In [ ]:
base = tf.keras.applications.ResNet50(include_top=False, input_shape=(48,48,1), weights=None)

x_ = tf.keras.layers.GlobalAveragePooling2D()(base.output)
output = tf.keras.layers.Dense(7, activation='softmax')(x_)

model = tf.keras.Model(inputs=base.input, outputs=output)

In [20]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_x, y_train, validation_data=(np.array(x_valid), np.array(y_valid)), epochs=10)

In [21]:
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

epochs = range(1, len(training_loss) + 1)

plt.plot(epochs, training_loss, 'r-')
plt.plot(epochs, validation_loss, 'b-')
plt.legend(['Training Loss', 'Validation Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Epochs vs Loss')
plt.show()

In [22]:
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

plt.plot(epochs, training_accuracy, 'r-')
plt.plot(epochs, validation_accuracy, 'b-')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Epochs vs Accuracy')
plt.show()

In [14]:
model.save('/dbfs/FileStore/tables/Models/vX')